# UPEC population structure and antimicrobial susceptibility in Norfolk, UK

### Globals

In [6]:
import glob
import re
from os import listdir, path, rename, system

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from Bio import SeqIO
from scipy.stats import chi2_contingency, fisher_exact, hypergeom
from statsmodels.stats.multitest import fdrcorrection


def rename_isolate(i):
    if i == "Reference":
        return i
    try:
        return re.search("GL\d+", i).group(0)
    except AttributeError:
        *_, GL = i.split("-")
        GL = "GL" + GL
        return rename_isolate(GL)

metadata = pd.read_excel('../Table S1.xlsx')

### Phylogroups

In [7]:
phylogroups = metadata['phylogroup']
phylogroups.value_counts(),phylogroups.value_counts(normalize=True).mul(100).round(1)

(B2    152
 D      28
 B1     19
 A       6
 F       6
 E       2
 C       2
 G       2
 Name: phylogroup, dtype: int64,
 B2    70.0
 D     12.9
 B1     8.8
 A      2.8
 F      2.8
 E      0.9
 C      0.9
 G      0.9
 Name: phylogroup, dtype: float64)

### Sequence types

In [8]:
st = metadata[['ID','ST']]
st = st.fillna(0)
st['ST'] = st['ST'].astype('int32')

In [9]:
st['ST'].value_counts()

73      28
12      20
69      19
131     18
404     14
        ..
372      1
108      1
3177     1
357      1
104      1
Name: ST, Length: 75, dtype: int64

In [13]:
str(round(st['ST'].value_counts()[:8].sum() / 217 * 100,1)) + '%'

'57.1%'

In [11]:
st['ST'].value_counts()[:8], st['ST'].value_counts(normalize=True).mul(100).round(1)[:8]

(73      28
 12      20
 69      19
 131     18
 404     14
 95      12
 127      7
 1193     6
 Name: ST, dtype: int64,
 73      12.9
 12       9.2
 69       8.8
 131      8.3
 404      6.5
 95       5.5
 127      3.2
 1193     2.8
 Name: ST, dtype: float64)

### Core genome alignment summary

In [16]:
# core alignment summary
ca = pd.read_csv('core_alignment_summary.txt', delimiter='\t')
ca = ca[:-1]
(ca['ALIGNED'] / ca['LENGTH']).describe()

count    217.000000
mean       0.843964
std        0.055073
min        0.728436
25%        0.782479
50%        0.858270
75%        0.890677
max        0.941170
dtype: float64

### AMR rates

In [14]:
amr = metadata[['ID', 'Source', 'AMP', 'AUG', 'CLX', 'CIP', 'GEN', 'NIT', 'TRI']]
amr = amr.iloc[18:,1:]

rx_rates = {
    "ABX" : [],
    "Overall" : [],
    "Community" : [],
    "Hospital" : []
    }

for col in amr.columns[1:]:
    rx_rates['ABX'].append(col)
    rx_rates['Overall'].append(amr.loc[:,col].value_counts(normalize=True)['R'].round(3) * 100)
    rx_rates['Hospital'].append(amr.loc[amr["Source"]=='Hospital',col].value_counts(normalize=True)['R'].round(3) * 100)
    try:
        rx_rates['Community'].append(amr.loc[amr["Source"]=='Community',col].value_counts(normalize=True)['R'].round(3) * 100)
    except KeyError:
        rx_rates['Community'].append(0)

pd.DataFrame(rx_rates)


,ABX,Overall,Community,Hospital
0,AMP,39.2,27.6,52.1
1,AUG,7.6,4.8,10.9
2,CLX,6.0,1.9,10.6
3,CIP,9.0,7.6,10.6
4,GEN,7.0,1.0,13.8
5,NIT,1.0,0.0,2.1
6,TRI,25.1,15.2,36.2


### Relationship between antibiotic resistance and source

In [26]:
amr = metadata[['ID', 'Source', 'AMP', 'AUG', 'CLX', 'CIP', 'GEN', 'NIT', 'TRI']]

def build_matrix_and_text(pheno):
    res_com = len(amr[(amr[pheno] == 'R') & (amr['Source']=='Community')])
    res_hosp = len(amr[(amr[pheno]== 'R') & (amr['Source']=='Hospital')])
    sens_com = len(amr[(amr[pheno] == 'S') & (amr['Source']=='Community')])
    sens_hosp = len(amr[(amr[pheno] == 'S') & (amr['Source']=='Hospital')])
    table = np.array([[res_com, res_hosp], [sens_com, sens_hosp]])
    chi2, p_chi, dof, _ = chi2_contingency(table, correction=False)
    return res_com, res_hosp, sens_com, sens_hosp, chi2, p_chi, dof

data = []
for r in ['AMP', 'AUG',	'CLX', 'CIP', 'GEN', 'NIT', 'TRI']:
    result = build_matrix_and_text(r)
    data.append([r]+list(result))

pd.DataFrame(data, columns=['Pheno', "res_com", 'res_hosp', 'sens_com', 'sens_hosp', 'chi2', 'p_chi', 'dof'])


,Pheno,res_com,res_hosp,sens_com,sens_hosp,chi2,p_chi,dof
0,AMP,29,49,76,45,12.500509,0.000407,1
1,AUG,5,10,100,82,2.600342,0.106841,1
2,CLX,2,10,103,84,6.676173,0.009771,1
3,CIP,8,10,97,84,0.549563,0.458496,1
4,GEN,1,13,104,81,12.575558,0.000391,1
5,NIT,0,2,105,92,2.256723,0.133035,1
6,TRI,16,34,89,60,11.551551,0.000677,1


### AMR determinants associated with phenotypic resistance

In [27]:
gene_pheno = pd.read_csv("gene_name_assoc.csv", index_col=0)

gene_pres_abs = pd.read_excel("AMR_genes_pres_abs.xlsx", index_col=0)
gene_pres_abs = gene_pres_abs.append(pd.Series(name='GL234', data=[0]*81, index=gene_pres_abs.columns, dtype=int))

amr = metadata[['ID', 'AMP', 'AUG', 'CLX', 'CIP', 'GEN', 'NIT', 'TRI']]
amr.index = amr['ID']
amr = amr.drop(columns='ID')

In [30]:
pheno_genes = {pheno : [] for pheno in amr.columns[1:]}
for gene, pheno1, pheno2, pheno3 in gene_pheno.itertuples():
    pheno_genes[pheno1].append(gene)
    if not isinstance(pheno2, float):
        pheno_genes[pheno2].append(gene)
    if not isinstance(pheno3, float):
        pheno_genes[pheno3].append(gene)

In [63]:
def build_matrix_and_text(pheno, geno):
    res_gene = len(amr[(amr[pheno] == 'R') & (amr.index.isin(gene_pres_abs[gene_pres_abs[geno]==1].index))])
    res_no_gene = len(amr[(amr[pheno]== 'R') & (~amr.index.isin(gene_pres_abs[gene_pres_abs[geno]==1].index))])
    sens_gene = len(amr[(amr[pheno] == 'S') & (amr.index.isin(gene_pres_abs[gene_pres_abs[geno]==1].index))])
    sens_no_gene = len(amr[(amr[pheno] == 'S') & (~amr.index.isin(gene_pres_abs[gene_pres_abs[geno]==1].index))])
    table = np.array([[res_gene, res_no_gene], [sens_gene, sens_no_gene]])
    oddsr, p_fisher = fisher_exact(table, alternative='two-sided')
    chi2, p_chi, dof, _ = chi2_contingency(table, correction=False)
    return res_gene, res_no_gene, sens_gene, sens_no_gene, oddsr, p_fisher, chi2, p_chi, dof

def get_cramer_assoc(p):
    phi2 = p / 217
    cramer_assoc = np.sqrt(phi2)
    return cramer_assoc

In [72]:
gene_pheno_assoc = {
    "gene" : [],
    "pheno" : [],
    "gene_pres_pheno_pos" : [],
    "gene_abs_pheno_pos" : [],
    "gene_pres_pheno_neg" : [],
    "gene_abs_pheno_neg" : [],
    "oddsr" : [],
    "p-value" : [],
    "chi2" : [],
    "p_chi" : [],
    "dof" : []
    }
for pheno,genes in pheno_genes.items():
    for gene in genes:
        result = build_matrix_and_text(pheno, gene)
        gene_pheno_assoc["gene"].append(gene)
        gene_pheno_assoc["pheno"].append(pheno)
        for key, val in zip(list(gene_pheno_assoc.keys())[2:], result):
            gene_pheno_assoc[key].append(val)

In [74]:
fet = pd.DataFrame(gene_pheno_assoc)
fet['cramer_assoc'] = fet['chi2'].apply(get_cramer_assoc)

p_value_corr = fdrcorrection(fet['p-value'])
fet = fet.assign(H0=p_value_corr[0], p_corrected=p_value_corr[1])

p_value_corr = fdrcorrection(fet['p_chi'])
fet = fet.assign(H0_chi=p_value_corr[0], p_chi_corrected=p_value_corr[1])

fet[(fet['H0']) & (fet['H0_chi'])].sort_values(by=['pheno', 'cramer_assoc'], ascending=False)


### Nitrofurantoin resistance determinants

In [79]:
columns = ['qaccver', 'saccver', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'sallseqid', 'score', 'nident', 'positive', 'gaps', 'ppos', 'qframe', 'sframe', 'qseq', 'sseq', 'qlen', 'slen', 'salltitles']
blastp = pd.read_csv('NIT_blastp_search.tabular', delimiter='\t', names=columns)
gpa = pd.read_csv('gene_presence_absence.csv')
ID_key_dict = gpa.filter(like='GL').iloc[0].to_dict()

In [96]:
NIT_seqs = []
directory = r"NITREc_Seq"
for f in listdir(directory):
    if f.endswith('.faa'):
        with open(rf"NITREc_Seq\{f}") as handle:
            for record in SeqIO.parse(handle, 'fasta'):
                NIT_seqs.append({
                    "gene" : f.replace('.faa', ''),
                    "description" : record.description,
                    "seq" : str(record.seq)
                })

In [95]:
def fetch_gene_info(seq):
    for dic in NIT_seqs:
        if seq == dic['seq']:
            return dic['gene'], dic['description']
            
def rename_value(value):
    return re.search('[A-Z]*', value).group(0)

In [89]:
map_rename = lambda x : mapped_IDs[rename_value(x)]
mapped_IDs = {rename_value(v) : k for k,v in ID_key_dict.items()}
blastp = blastp.drop(blastp[blastp['saccver'].str.startswith('HNPKFMLJ')].index)
blastp["isolate"] = blastp['saccver'].map(map_rename)
blastp[['gene', 'description']] = pd.DataFrame(blastp['qseq'].map(fetch_gene_info).to_list(), index=blastp.index)
blastp = blastp[(blastp['ppos'] == 100) & (blastp['pident'] == 100) & (blastp['qlen'] == blastp['slen'])]


In [94]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(blastp[blastp['isolate'].isin(amr[amr['NIT']=='R'].index)][['isolate', 'description', 'gene']])

      isolate                                        description     gene
1342    GL116  NfsA nfsA|IN09|2|287989-288480|-|ICL2020_00865...  NfsA__R
5150     GL14  NfsA nfsA|EC958|NZ_HG941718.1|900522-901244|+|...  NfsA__S
10266     GL4  NfsB nfsB|IN06|1|705493-705774|+|ICL2020_00712...  NfsB__R
10619   GL160  NfsB nfsB|IN07|2|122188-122619|+|ICL2020_00466...  NfsB__R
15019    GL14  NfsB nfsB|ATCC25922|NZ_CP009072.1|4640270-4640...  NfsB__S
18477    GL14  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18479    GL13  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18484    GL12  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18488     GL3  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18489    GL11  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18494     GL2  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18498     GL5  RibE ribE|ATCC25922|NZ_CP009072.1|4779388-4779...  RibE__S
18501     GL4  RibE ribE|ATCC25922|NZ_